# All oscillation analysis

Brief 1-2 sentence description of notebook.

In [1]:
# Imports of all used packages and libraries
import sys
import os
import git
import glob
from collections import defaultdict

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'C:/Users/ryoiwata/Documents/GitHub/reward_comp_ext'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import os
import collections
import itertools
from collections import defaultdict
from itertools import combinations

In [7]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import mannwhitneyu
# import seaborn as sns



In [8]:
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import colorsys

In [9]:
from sklearn.metrics import confusion_matrix

In [10]:
# from spectral_connectivity import Multitaper, Connectivity
# import spectral_connectivity

In [11]:
import utilities.helper
import trodes.read_exported

In [12]:
FONTSIZE = 20

In [13]:
font = {'weight' : 'medium',
        'size'   : 20}

matplotlib.rc('font', **font)

In [14]:
# Define a function to horizontally stack arrays
def stack_arrays(arrays):
    return np.vstack(arrays)

In [15]:
# BAND_TO_FREQ_PLOT = {'theta': (4, 12), 'gamma': (30, 50)}
BAND_TO_FREQ_PLOT = {'theta': (4, 12)}
BAND_TO_FREQ_COLOR = {'theta': "#FFAF00", 'beta': "blue", 'gamma': "green"}

## Inputs & Data

In [16]:
EPHYS_SAMPLE_RATE = 20000

In [17]:
# GOOD_SUBJECTS = ["3.1", "3.3", "3.4", "4.2", "4.3", "5.2", "5.3"]
# GOOD_SUBJECTS = ["3.1", "4.2", "4.3"]
# GOOD_SUBJECTS = ["3.1", "3.3", "3.4", "4.2", "4.3"]
GOOD_SUBJECTS = ["3.1", "3.3", "3.4", "4.2", "4.3", "5.2", "5.3"]
# GOOD_SUBJECTS = ["3.1", "3.3", "3.4", "4.2", "4.3"]


Explanation of each input and where it comes from.

In [18]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIXES = ["rce_pilot_3_alone_comp", "rce_pilot_3_long_comp"]
OUTPUT_PREFIX = "rce_pilot_3_combined"

In [19]:
# TRIAL_LABELS_DF = pd.read_excel("/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/reward_comp_ext/results/2024_06_26_sleap_clustering/data/rce_pilot_3_alone_comp_per_video_trial_labels.xlsx")
TRIALS_AND_SPECTRAL_DF = pd.concat([pd.read_pickle("./proc/{}_10_per_trial_spectral_bans_sleap.pkl".format(prefix)) for prefix in OUTPUT_PREFIXES])

In [20]:
FULL_LFP_TRACES_PKL = "{}_12_per_cluster_spectral_bans_sleap.pkl".format(OUTPUT_PREFIX)

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Functions 

In [21]:
def combine_dicts(dicts):
    """
    Combine lists from multiple dictionaries that share the same key.

    This function takes a list of dictionaries where each dictionary's values are lists of numbers.
    It merges these lists for each corresponding key across all dictionaries, producing a single
    dictionary where each key has a combined list of all numbers from the input dictionaries.

    Parameters:
        dicts (list of dict): A list of dictionaries with values as lists of numbers.

    Returns:
        dict: A dictionary with keys from the input dictionaries and values as merged lists 
        of numbers from all corresponding input dictionary values.
    """
    combined = defaultdict(list)
    for dictionary in dicts:
        for key, value in dictionary.items():
            combined[key].extend(value)
    
    return dict(combined)


# Example usage
list_of_dicts = [
    {'a': [1, 2], 'b': [3, 4]},
    {'a': [5], 'b': [6, 7]},
    {'a': [8, 9], 'c': [10]}
]

combined_dict = combine_dicts(list_of_dicts)
print(combined_dict)

{'a': [1, 2, 5, 8, 9], 'b': [3, 4, 6, 7], 'c': [10]}


In [22]:
def find_consecutive_ranges(numbers, min_length=1):
    """
    Finds the start and end indices for consecutive ranges of each number in a list where the range meets a minimum length.

    This function iterates through a list of integers and identifies ranges where the same integer appears consecutively
    and the length of this sequence meets or exceeds the specified minimum length.

    Parameters:
        numbers (list): A list of integers to analyze for consecutive ranges.
        min_length (int): The minimum length of a range for it to be included in the results.

    Returns:
        dict: A dictionary with integers as keys and a list of tuples (start, end) as values,
              where each tuple represents the start and end indices (inclusive) of consecutive ranges
              for that integer. Only ranges that meet or exceed the minimum length are included.
    """
    ranges = {}
    n = len(numbers)
    if n == 0:
        return ranges
    
    start = 0
    current = numbers[0]

    for i in range(1, n):
        if numbers[i] != current:
            if (i - start) >= min_length:
                if current not in ranges:
                    ranges[current] = []
                ranges[current].append((start, i - 1))
            current = numbers[i]
            start = i

    # Handle the last range
    if (n - start) >= min_length:
        if current not in ranges:
            ranges[current] = []
        ranges[current].append((start, n - 1))

    return ranges

# Example usage:
numbers = [1, 1, 2, 2, 2, 3, 3, 3, 3, 2, 2, 1, 1]
print(find_consecutive_ranges(numbers, min_length=3))

{2: [(2, 4)], 3: [(5, 8)]}


In [23]:
def update_tuples_in_dict(original_dict, reference_list):
    """
    Updates the values in the tuples within a dictionary by replacing indices with corresponding values from a reference list.
    
    This function iterates through each key-value pair in the original dictionary. Each value is expected to be a list of tuples,
    where each tuple contains indices. These indices are used to fetch corresponding values from the reference list, creating new tuples.
    
    Parameters:
        original_dict (dict): Dictionary whose values are lists of tuples. Each tuple consists of indices into the reference_list.
        reference_list (list): List of elements that are referenced by the indices in the tuples of the original_dict.
        
    Returns:
        dict: A dictionary with the same keys as original_dict but with tuples transformed to contain elements from reference_list
              based on the indices in the original tuples.
    """
    # Create a new dictionary to store the updated key-value pairs
    new_dict = {}
    for key, list_of_tuples in original_dict.items():
        # Process each tuple in the list associated with the current key
        updated_tuples = [
            tuple(reference_list[idx] for idx in tup) for tup in list_of_tuples
        ]
        new_dict[key] = updated_tuples
    
    return new_dict

# Example usage:
original_dict = {
    'a': [(0, 1), (2, 3)],
    'b': [(1, 3), (0, 2)]
}
reference_list = ['alpha', 'beta', 'gamma', 'delta']

updated_dict = update_tuples_in_dict(original_dict, reference_list)
print(updated_dict)

{'a': [('alpha', 'beta'), ('gamma', 'delta')], 'b': [('beta', 'delta'), ('alpha', 'gamma')]}


In [24]:
# def find_indices_within_ranges(ranges_dict, values):
#     """
#     Creates a dictionary mapping keys to sorted indices of values that fall within specified ranges.
    
#     Parameters:
#         ranges_dict (dict): A dictionary with keys and values as lists of tuples representing ranges.
#         values (list): A list of values to check against the ranges.
        
#     Returns:
#         dict: A dictionary where each key maps to a sorted list of indices for values within the ranges.
#     """
#     result_dict = {}
#     for key, ranges in ranges_dict.items():
#         matched_indices = []
#         for index, value in enumerate(values):
#             if any(start <= value <= end for start, end in ranges):
#                 matched_indices.append(index)
#         result_dict[key] = sorted(matched_indices)
#     return list(result_dict.items())

# # Example usage:
# ranges_dict = {
#     'range1': [(1, 5), (10, 15)],
#     'range2': [(0, 2), (4, 8)]
# }
# values = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

# result = find_indices_within_ranges(ranges_dict, values)
# print(result)


In [25]:
def update_tuples_in_list(original_list, reference_list):
    """
    Updates the values in the tuples within a list by replacing indices with corresponding values from a reference list.
    
    This function iterates through each tuple in the original list. Each tuple is expected to contain indices.
    These indices are used to fetch corresponding values from the reference list, creating new tuples.
    
    Parameters:
        original_list (list): List of tuples. Each inner tuple consists of indices into the reference_list.
        reference_list (list): List of elements that are referenced by the indices in the tuples of the original_list.
        
    Returns:
        list: A list with the same structure as original_list but with tuples transformed to contain elements from reference_list
              based on the indices in the original tuples.
    """
    # Create a new list to store the updated tuples
    new_list = [
        tuple(reference_list[idx] for idx in tup) for tup in original_list
    ]
    
    return new_list

# Example usage:
original_list = [
    (0, 1), (2, 3),
    (1, 3), (0, 2)
]
reference_list = ['alpha', 'beta', 'gamma', 'delta']

updated_list = update_tuples_in_list(original_list, reference_list)
print(updated_list)

[('alpha', 'beta'), ('gamma', 'delta'), ('beta', 'delta'), ('alpha', 'gamma')]


In [26]:
def find_indices_within_ranges(ranges_list, values):
    """
    Finds the indices of values that fall within specified ranges.
    
    Parameters:
        ranges_list (list): A list of tuples representing ranges.
        values (list): A list of values to check against the ranges.
        
    Returns:
        list: A list of sorted indices for values within the ranges.
    """
    matched_indices = []
    for index, value in enumerate(values):
        if any(start <= value <= end for start, end in ranges_list):
            matched_indices.append(index)
    
    return sorted(matched_indices)

# Example usage:
ranges_list = [(1, 5), (10, 15), (0, 2), (4, 8)]
values = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

result = find_indices_within_ranges(ranges_list, values)
print(result)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15]


In [27]:
OUTCOME_TO_COLOR = {"both_rewarded": "green", 
"novel_lose": "#e67073",
"novel_win": "#93a5da",
"lose": "#951a1d",
"alone_rewarded": "#0499af",
"win": "#3853a3",
"omission": "orange",
"tie": "green"}

In [28]:
comp_id_to_color = {'competitive_1': "#281640",
'competitive_2': "#43246a",
'competitive_3': "#8e7ca6",
'no_comp_4': "#2f3600",
'no_comp_5': "#535f00",
'no_comp_6': "#768800",
'no_comp_7': "#9fac4d",
'no_comp_8': "#c8cf99",
'competitive': "#43246A",
'no_comp': "#768800",
'win': "#0045A6",
'win_competitive': "#003074",
'win_no_comp': "#4d7dc1",
'lose': "#792910",
'lose_competitive': "#551d0b",
'lose_no_comp': "#a16958",
'rewarded': "#FFAF00"
}

In [29]:
to_keep_columns = ['trial_label',
'tone_start_frame',
'reward_start',
'reward_dispensed',
'tone_stop_frame',
'condition',
'competition_closeness',
'get_reward_frame',
'out_reward_frame',
'notes',
'box_1_port_entry_frames',
'box_2_port_entry_frames',
'video_name',
'tone_start_timestamp',
'tone_stop_timestamp',
'box_1_port_entry_timestamps',
'box_2_port_entry_timestamps',
'current_subject',
'session_dir',
'experiment',
'sleap_name',
'video_id',
'agent',
'all_subjects',
'cohort',
'first_timestamp',
'last_timestamp',
'recording',
'session_path',
'subject',
'baseline_start_timestamp',
'post_trial_end_timestamp',]

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [30]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# Ideally functions are defined here first and then data is processed using the functions

# function names are short and in snake case all lowercase
# a function name should be unique but does not have to describe the function
# doc strings describe functions not function names




## Renaming the trial labels

In [31]:
TRIALS_AND_SPECTRAL_DF.head()

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,experiment,video_name,...,baseline_mPFC_gamma_phase,baseline_mPFC_lfp_trace,baseline_mPFC_theta_band,baseline_mPFC_theta_phase,baseline_vHPC_gamma_band,baseline_vHPC_gamma_phase,baseline_vHPC_lfp_trace,baseline_vHPC_theta_band,baseline_vHPC_theta_phase,baseline_lfp_timestamps
0,1272,1352,1372.0,1471,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[-2.6797059, -2.3968484, -2.1131911, -1.828970...","[45.434998, 53.039997, 63.569996, 94.575, 114....","[0.3023039, 0.3250822, 0.34729052, 0.3688477, ...","[-0.788324, -0.7305608, -0.67332137, -0.616575...","[0.11822242, 0.026445314, -0.061667122, -0.141...","[1.2789538, 1.5046133, 1.7300067, 1.9551437, 2...","[-4.875, -0.39, 26.519999, 62.399998, 69.615, ...","[0.45442012, 0.4403637, 0.4263141, 0.4122978, ...","[0.5867341, 0.61907744, 0.6510671, 0.682684, 0...","[1075900, 1075920, 1075940, 1075960, 1075980, ..."
1,1272,1352,1372.0,1471,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[-1.0192864, -0.7884174, -0.5577601, -0.327317...","[232.635, 241.79999, 253.49998, 259.935, 290.5...","[0.86803913, 0.8516945, 0.83444965, 0.8164087,...","[0.042734537, 0.07712564, 0.1110838, 0.1445905...","[0.04515997, 0.09145398, 0.13580361, 0.1756941...","[-1.3614203, -1.1462942, -0.92886734, -0.70931...","[221.715, 238.09499, 259.935, 264.03, 253.305,...","[0.49248752, 0.49964046, 0.50668645, 0.5136666...","[-0.7678083, -0.7481149, -0.7286564, -0.709394...","[1075900, 1075920, 1075940, 1075960, 1075980, ..."
2,3268,3348,3368.0,3469,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[0.032740027, 0.27976346, 0.52697045, 0.774348...","[128.50499, 109.979996, 77.415, 22.619999, -20...","[0.31883484, 0.304557, 0.28981718, 0.27463654,...","[0.82180655, 0.86382616, 0.90566295, 0.9473232...","[-0.1966364, -0.20901106, -0.2086731, -0.19691...","[2.5922303, 2.8197713, 3.0456688, -3.0136209, ...","[-46.995, -35.489998, -35.684998, -43.484997, ...","[0.24851131, 0.20672616, 0.16414632, 0.1208212...","[1.333335, 1.3743384, 1.4155797, 1.4570614, 1....","[3075920, 3075940, 3075960, 3075980, 3076000, ..."
3,3268,3348,3368.0,3469,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[1.9588519, 2.221262, 2.4844332, 2.7484064, 3....","[56.55, -39.195, -152.09999, -172.185, -144.10...","[0.019023584, 0.033879466, 0.04848842, 0.06283...","[-1.5093546, -1.4610933, -1.4133036, -1.365974...","[-0.034169298, -0.058782697, -0.0758774, -0.08...","[1.9731913, 2.3358655, 2.7034516, 3.0720255, -...","[0.78, 53.82, 101.59499, 119.729996, 94.77, 49...","[-0.0010840435, 0.03636438, 0.0744549, 0.11305...","[-1.572606, -1.5110681, -1.450328, -1.3903525,...","[3075920, 3075940, 3075960, 3075980, 3076000, ..."
4,4466,4546,4566.0,4667,rewarded,NaN,NaN,NaN,alone_comp,20240320_142408_alone_comp_subj_3-1_and_3-3.1,...,"[2.3436375, 2.551059, 2.757349, 2.9625306, -3....","[60.644997, 38.805, 30.81, 48.554996, 77.805, ...","[0.60134304, 0.6429786, 0.68366385, 0.7233099,...","[-1.0260117, -0.98588574, -0.9458348, -0.90586...","[-0.031517684, 0.033092283, 0.09645234, 0.1545...","[-1.6889625, -1.4474249, -1.2057016, -0.963856...","[223.665, 226.39499, 232.04999, 241.995, 243.1...","[1.0194497, 1.0016606, 0.98306626, 0.9636855, ...","[0.47053403, 0.5033234, 0.53599346, 0.56855184...","[4275940, 4275960, 4275980, 4276000, 4276020, ..."


In [32]:
comp_closeness_dict = {'Subj 1 blocking Subj 2': "competitive",
'Subj 2 Only': "no_comp",
'Subj 2 blocking Subj 1': "competitive",
'Subj 1 then Subj 2': "competitive", 
'Subj 1 Only': "no_comp",
'Subj 2 then Subj 1': "competitive",
'Close Call': "competitive",
'After trial': "no_comp"}

In [33]:
# cluster_to_competitiveness = {"0": "no_comp", "1": "competitive", "2": "competitive", "3": "no_comp", "4": "competitive", "5": "no_comp", "6": "no_comp", "7": "no_comp"}
# cluster_to_comp_id = {"0": "no_comp_8", "1": "competitive_3", "2": "competitive_1", "3": "no_comp_6", "4": "competitive_2", "5": "no_comp_7", "6": "no_comp_5", "7": "no_comp_4"}
# comp_id = {"no_comp_8", "competitive_3", "competitive_1", "no_comp_6", "competitive_2", "no_comp_7", "no_comp_5", "no_comp_4"}


Win base color
#0045A6 
Win competitive color
#003074
Win no comp color
#4d7dc1

Lose base color
#792910
Lose competitive color
#551d0b
Lose no comp color
#a16958

In [34]:
TRIALS_AND_SPECTRAL_DF["current_subject"] = TRIALS_AND_SPECTRAL_DF["current_subject"].apply(lambda x: str(x).strip().lower())

In [35]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["current_subject"].isin(GOOD_SUBJECTS)]

In [36]:
TRIALS_AND_SPECTRAL_DF["current_subject"].unique()

array(['3.1', '3.3', '4.2', '4.3', '3.4', '5.2', '5.3'], dtype=object)

In [37]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["condition"] != "tie"].reset_index(drop=True)

In [38]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["condition"] != "temp"].reset_index(drop=True)

In [39]:
TRIALS_AND_SPECTRAL_DF["condition"].unique()

array(['rewarded', 3.1, 3.3, 4.3, 3.4, 4.4, 5.3, 4.2, 5.4, 5.2],
      dtype=object)

In [40]:
TRIALS_AND_SPECTRAL_DF["condition"] = TRIALS_AND_SPECTRAL_DF["condition"].apply(lambda x: str(x).strip().lower())

In [41]:
TRIALS_AND_SPECTRAL_DF["trial_label"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: "win" if x["current_subject"] == x["condition"]  else ("lose" if x["agent"] == x["condition"] else x["condition"]), axis=1)
                                                                        

In [42]:
TRIALS_AND_SPECTRAL_DF["trial_label"].unique()

array(['rewarded', 'win', 'lose'], dtype=object)

In [43]:
TRIALS_AND_SPECTRAL_DF["competition_closeness"] = TRIALS_AND_SPECTRAL_DF["competition_closeness"].map(comp_closeness_dict).fillna("rewarded")

In [44]:
TRIALS_AND_SPECTRAL_DF["trial_label"].unique()

array(['rewarded', 'win', 'lose'], dtype=object)

In [45]:
TRIALS_AND_SPECTRAL_DF["competition_closeness"].unique()

array(['rewarded', 'no_comp', 'competitive'], dtype=object)

## Making separate rows for each cluster

In [46]:
TRIALS_AND_SPECTRAL_DF["competitiveness_frame_ranges_dict"]

0                                                     NaN
1                                                     NaN
2                                                     NaN
3                                                     NaN
4                                                     NaN
                              ...                        
1138    {'competitive': [(50311, 50319), (50462, 50491...
1139    {'no_comp': [(51709, 51822), (52220, 52250), (...
1140    {'no_comp': [(51709, 51822), (52220, 52250), (...
1141    {'competitive': [(53904, 53948), (53980, 54023...
1142    {'competitive': [(53904, 53948), (53980, 54023...
Name: competitiveness_frame_ranges_dict, Length: 1143, dtype: object

In [47]:
TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["trial_label"] != "rewarded"]["competitiveness_frame_ranges_dict"].iloc[0]

{np.str_('competitive'): [(np.int64(34040), np.int64(34045)),
  (np.int64(34241), np.int64(34257)),
  (np.int64(34264), np.int64(34298)),
  (np.int64(34433), np.int64(34536)),
  (np.int64(34545), np.int64(34639))],
 np.str_('no_comp'): [(np.int64(34046), np.int64(34240)),
  (np.int64(34258), np.int64(34263)),
  (np.int64(34299), np.int64(34432)),
  (np.int64(34537), np.int64(34544))]}

In [48]:
TRIALS_AND_SPECTRAL_DF["tone_start_frame"].head()

0    1272
1    1272
2    3268
3    3268
4    4466
Name: tone_start_frame, dtype: int64

In [49]:
TRIALS_AND_SPECTRAL_DF["tone_stop_frame"]

0        1471
1        1471
2        3469
3        3469
4        4667
        ...  
1138    50711
1139    52108
1140    52108
1141    54304
1142    54304
Name: tone_stop_frame, Length: 1143, dtype: int64

- Making each dictionary into a list so that we can explode it

In [50]:
dict_col = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "dict" in col and "competitiveness" in col]


In [51]:
dict_col

['competitiveness_index_ranges_dict', 'competitiveness_frame_ranges_dict']

In [52]:
TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["trial_label"] != "rewarded"]["competitiveness_index_ranges_dict"].iloc[0]

{np.str_('competitive'): [(0, 5),
  (201, 217),
  (224, 258),
  (393, 496),
  (505, 599)],
 np.str_('no_comp'): [(6, 200), (218, 223), (259, 392), (497, 504)]}

In [53]:
for col in dict_col:
    TRIALS_AND_SPECTRAL_DF[col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: list(x[col].items()) if isinstance(x[col], dict) else [("rewarded", [(x["tone_start_frame"], x["tone_stop_frame"])])], axis=1)

In [54]:
TRIALS_AND_SPECTRAL_DF["competitiveness_frame_ranges_dict"].head()

0    [(rewarded, [(1272, 1471)])]
1    [(rewarded, [(1272, 1471)])]
2    [(rewarded, [(3268, 3469)])]
3    [(rewarded, [(3268, 3469)])]
4    [(rewarded, [(4466, 4667)])]
Name: competitiveness_frame_ranges_dict, dtype: object

In [55]:
TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["trial_label"] != "rewarded"]["competitiveness_frame_ranges_dict"].head()

42    [(competitive, [(np.int64(34040), np.int64(340...
43    [(competitive, [(np.int64(34040), np.int64(340...
44    [(no_comp, [(np.int64(35038), np.int64(35090))...
45    [(no_comp, [(np.int64(35038), np.int64(35090))...
46    [(no_comp, [(np.int64(36137), np.int64(36386))...
Name: competitiveness_frame_ranges_dict, dtype: object

- Exploding each row so that competitive and non-competitive clusters are on different rows

In [56]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.explode(column=dict_col)

In [57]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF["competitiveness_frame_ranges_dict"].head()

0    (rewarded, [(1272, 1471)])
1    (rewarded, [(1272, 1471)])
2    (rewarded, [(3268, 3469)])
3    (rewarded, [(3268, 3469)])
4    (rewarded, [(4466, 4667)])
Name: competitiveness_frame_ranges_dict, dtype: object

In [58]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF[cluster_exploded_TRIALS_AND_SPECTRAL_DF["trial_label"] != "rewarded"]["competitiveness_frame_ranges_dict"].head()

42    (competitive, [(34040, 34045), (34241, 34257),...
42    (no_comp, [(34046, 34240), (34258, 34263), (34...
43    (competitive, [(34040, 34045), (34241, 34257),...
43    (no_comp, [(34046, 34240), (34258, 34263), (34...
44    (no_comp, [(35038, 35090), (35167, 35172), (35...
Name: competitiveness_frame_ranges_dict, dtype: object

- Making a new column based on the key which is the competitiveness of the group

In [59]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF["competitiveness_label"] = cluster_exploded_TRIALS_AND_SPECTRAL_DF["competitiveness_frame_ranges_dict"].apply(lambda x: str(x[0]))

In [60]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF["competitiveness_label"].iloc[0]

'rewarded'

In [61]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF["competitiveness_label"].head()

0    rewarded
1    rewarded
2    rewarded
3    rewarded
4    rewarded
Name: competitiveness_label, dtype: object

In [62]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF[cluster_exploded_TRIALS_AND_SPECTRAL_DF["trial_label"] != "rewarded"]["competitiveness_label"].head()

42    competitive
42        no_comp
43    competitive
43        no_comp
44        no_comp
Name: competitiveness_label, dtype: object

- Filtering rows that don't have matching frame and trial competitiveness

In [63]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF["competitiveness_label"].unique()

array(['rewarded', 'competitive', 'no_comp'], dtype=object)

In [64]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF["competition_closeness"].unique()

array(['rewarded', 'no_comp', 'competitive'], dtype=object)

In [65]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF.shape

(1752, 410)

In [66]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF = cluster_exploded_TRIALS_AND_SPECTRAL_DF[cluster_exploded_TRIALS_AND_SPECTRAL_DF["competition_closeness"] == cluster_exploded_TRIALS_AND_SPECTRAL_DF["competitiveness_label"]].reset_index(drop=True)

In [67]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF.shape

(1143, 410)

- Combining the competitiveness and the trial outcome labels

In [68]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF["trial_and_competitiveness_label"] = cluster_exploded_TRIALS_AND_SPECTRAL_DF.apply(lambda x: "_".join([x["trial_label"], x["competitiveness_label"]]) if  x["trial_label"] != x["competitiveness_label"] else x["trial_label"], axis=1)

In [69]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF["trial_and_competitiveness_label"].unique()

array(['rewarded', 'win_no_comp', 'lose_no_comp', 'win_competitive',
       'lose_competitive'], dtype=object)

- Making a new column that is the range of frames

In [70]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF["competitiveness_frame_ranges"] = cluster_exploded_TRIALS_AND_SPECTRAL_DF["competitiveness_frame_ranges_dict"].apply(lambda x: x[1])

In [71]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF["competitiveness_frame_ranges"].head()

0    [(1272, 1471)]
1    [(1272, 1471)]
2    [(3268, 3469)]
3    [(3268, 3469)]
4    [(4466, 4667)]
Name: competitiveness_frame_ranges, dtype: object

In [72]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF[cluster_exploded_TRIALS_AND_SPECTRAL_DF["trial_label"] != "rewarded"]["competitiveness_frame_ranges"].head()

42    [(34046, 34240), (34258, 34263), (34299, 34432...
43    [(34046, 34240), (34258, 34263), (34299, 34432...
44    [(35091, 35166), (35173, 35200), (35249, 35334...
45    [(35091, 35166), (35173, 35200), (35249, 35334...
46    [(36137, 36386), (36438, 36503), (36509, 36516...
Name: competitiveness_frame_ranges, dtype: object

- Filtering for ranges that are only during the trial

In [73]:
def adjust_frame_range(frame_range, event_range):
    """
    Adjusts the frame range based on the event range.

    Parameters:
        frame_range (tuple): The tuple representing the start and end of the frame range.
        event_range (tuple): The tuple representing the start and end of the event range.

    Returns:
        tuple or None: Adjusted frame range or None if there's no overlap.
    """
    # Unpack the ranges
    frame_start, frame_end = frame_range
    event_start, event_end = event_range

    # Check for overlap
    if frame_end < event_start or frame_start > event_end:
        return None  # No overlap

    # Adjust the start and end of the frame range
    adjusted_start = max(frame_start, event_start)
    adjusted_end = min(frame_end, event_end)

    return (adjusted_start, adjusted_end)

# Example Usage:
frame_range = (10, 20)
event_range = (15, 25)
result = adjust_frame_range(frame_range, event_range)
print(result)  # Output will be (15, 20)


(15, 20)


In [74]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF["tone_start_frame"]

0        1272
1        1272
2        3268
3        3268
4        4466
        ...  
1138    50511
1139    51909
1140    51909
1141    54104
1142    54104
Name: tone_start_frame, Length: 1143, dtype: int64

In [75]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF["tone_stop_frame"]

0        1471
1        1471
2        3469
3        3469
4        4667
        ...  
1138    50711
1139    52108
1140    52108
1141    54304
1142    54304
Name: tone_stop_frame, Length: 1143, dtype: int64

In [76]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF["competitiveness_frame_ranges"] = cluster_exploded_TRIALS_AND_SPECTRAL_DF.apply(lambda x: [adjust_frame_range((start, stop), (x["tone_start_frame"], x["tone_stop_frame"])) for start, stop in x["competitiveness_frame_ranges"] if adjust_frame_range((start, stop), (x["tone_start_frame"], x["tone_stop_frame"]))], axis=1)

- Filtering for rows that have at least one consecutive second

In [77]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF = cluster_exploded_TRIALS_AND_SPECTRAL_DF[cluster_exploded_TRIALS_AND_SPECTRAL_DF["competitiveness_frame_ranges"].apply(lambda x: len(x) >= 1)]

- Filtering for ranges that are 1 second consecutive

In [78]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF["competitiveness_frame_ranges"] = cluster_exploded_TRIALS_AND_SPECTRAL_DF["competitiveness_frame_ranges"].apply(lambda x: [(start, stop) for start, stop in x if stop - start >= 20])

- Calculating the total number of frames for each trial

In [79]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF["frame_ranges_sum_frames"] = cluster_exploded_TRIALS_AND_SPECTRAL_DF["competitiveness_frame_ranges"].apply(lambda x: sum([stop - start for start, stop in x]))

In [80]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF["frame_ranges_sum_frames"]

0       199
1       199
2       201
3       201
4       201
       ... 
1138     69
1139    199
1140    199
1141    151
1142    151
Name: frame_ranges_sum_frames, Length: 1136, dtype: int64

- Filtering for rows where at least half the frames are there

In [81]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF = cluster_exploded_TRIALS_AND_SPECTRAL_DF[cluster_exploded_TRIALS_AND_SPECTRAL_DF["frame_ranges_sum_frames"] >= 100]

In [82]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF["competitiveness_frame_ranges"]

0                         [(1272, 1471)]
1                         [(1272, 1471)]
2                         [(3268, 3469)]
3                         [(3268, 3469)]
4                         [(4466, 4667)]
                      ...               
1136    [(49514, 49589), (49676, 49713)]
1139                    [(51909, 52108)]
1140                    [(51909, 52108)]
1141    [(54120, 54212), (54245, 54304)]
1142    [(54120, 54212), (54245, 54304)]
Name: competitiveness_frame_ranges, Length: 1010, dtype: object

- Filtering for rows that have at least one consecutive second

In [83]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF = cluster_exploded_TRIALS_AND_SPECTRAL_DF[cluster_exploded_TRIALS_AND_SPECTRAL_DF["competitiveness_frame_ranges"].apply(lambda x: len(x) >= 1)]

In [84]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF["competitiveness_frame_ranges"]

0                         [(1272, 1471)]
1                         [(1272, 1471)]
2                         [(3268, 3469)]
3                         [(3268, 3469)]
4                         [(4466, 4667)]
                      ...               
1136    [(49514, 49589), (49676, 49713)]
1139                    [(51909, 52108)]
1140                    [(51909, 52108)]
1141    [(54120, 54212), (54245, 54304)]
1142    [(54120, 54212), (54245, 54304)]
Name: competitiveness_frame_ranges, Length: 1010, dtype: object

In [85]:
for col in cluster_exploded_TRIALS_AND_SPECTRAL_DF.columns:
    print(col)

tone_start_frame
reward_start
reward_dispensed
tone_stop_frame
condition
competition_closeness
get_reward_frame
out_reward_frame
experiment
video_name
tone_start_timestamp
tone_stop_timestamp
cohort
session_dir
tone_frames
box_1_port_entry_frames
box_2_port_entry_frames
notes
session_path
recording
current_subject
subject
all_subjects
first_timestamp
last_timestamp
video_timestamps
tone_timestamps
box_1_port_entry_timestamps
box_2_port_entry_timestamps
power_calculation_frequencies
coherence_calculation_frequencies
granger_calculation_frequencies
tone_start_frame_remove
agent_tail_base
agent_to_reward_port_angle
subject_tail_base
subject_to_reward_port_angle
to_reward_port_angle_diff
to_reward_port_angle_sum
within_trial_index
agent
10s_after_tone_frame
10s_before_tone_frame
all_subjects_remove
body_parts
box_bottom_left
box_bottom_right
box_number
box_top_left
box_top_right
color
comp_id
competition_closeness_remove
condition 
experiment_remove
first_timestamp_remove
get_reward_frame_

## Adding the velocity

In [86]:
alone_trials_df = pd.read_pickle("../2024_06_26_sleap_clustering/proc/alone_trials_sleap.pkl")

In [87]:
alone_trials_df.columns

Index(['unnamed: 0', 'session_dir', 'tone_start_frame', 'reward_start',
       'reward_dispensed', 'tone_stop_frame', 'condition',
       'competition_closeness', 'get_reward_frame', 'out_reward_frame',
       'notes', 'experiment', 'box_1_port_entry_frames',
       'box_2_port_entry_frames', 'video_name', 'all_subjects',
       'first_timestamp', 'last_timestamp', 'tone_start_timestamp',
       'tone_stop_timestamp', 'box_1_port_entry_timestamps',
       'box_2_port_entry_timestamps', '10s_before_tone_frame',
       '10s_after_tone_frame', 'tone_start_to_stop_frame', 'start_frame',
       'stop_frame', 'tracked_subject', 'in_video_subjects', 'box_number',
       'sleap_name', 'current_subject', 'body_parts', 'box_top_right',
       'box_bottom_right', 'box_top_left', 'reward_port', 'box_bottom_left',
       'agent', 'subject_locations', 'agent_locations', 'frame_index',
       'subject_thorax', 'subject_nose', 'subject_tail_base', 'agent_thorax',
       'agent_nose', 'agent_tail_base'

In [88]:
alone_trials_df["subject_thorax_velocity"].apply(lambda x: x.shape)

0      (599,)
1      (601,)
2      (601,)
3      (599,)
4      (599,)
        ...  
289    (601,)
290    (599,)
291    (599,)
292    (600,)
293    (601,)
Name: subject_thorax_velocity, Length: 294, dtype: object

In [126]:
alone_trials_df["trial_subject_thorax_velocity"] = alone_trials_df["subject_thorax_velocity"].apply(lambda x: x[199:399])

In [127]:
alone_trials_df["trial_frame_index"] = alone_trials_df["frame_index"].apply(lambda x: x[199:399])

In [128]:
alone_trial_to_velocity = {(row['session_dir'], row['current_subject'], row['tone_start_timestamp']): row['trial_subject_thorax_velocity'] for index, row in alone_trials_df.iterrows()}

In [129]:
alone_trial_to_frame_index = {(row['session_dir'], row['current_subject'], row['tone_start_timestamp']): row['trial_frame_index'] for index, row in alone_trials_df.iterrows()}

In [130]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF["trial_subject_thorax_velocity"] = cluster_exploded_TRIALS_AND_SPECTRAL_DF.apply(lambda row: alone_trial_to_velocity[(row['session_dir'], row['current_subject'], row['tone_start_timestamp'])] if (row['session_dir'], row['current_subject'], row['tone_start_timestamp']) in alone_trial_to_velocity else row["trial_subject_thorax_velocity"], axis=1)

In [131]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF["trial_frame_index"] = cluster_exploded_TRIALS_AND_SPECTRAL_DF.apply(lambda row: alone_trial_to_frame_index[(row['session_dir'], row['current_subject'], row['tone_start_timestamp'])] if (row['session_dir'], row['current_subject'], row['tone_start_timestamp']) in alone_trial_to_frame_index else row["trial_frame_index"], axis=1)

In [132]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF["trial_subject_thorax_velocity"].iloc[:5].apply(lambda x: x.shape)

0    (200,)
1    (200,)
2    (200,)
3    (200,)
4    (200,)
Name: trial_subject_thorax_velocity, dtype: object

In [133]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF["trial_frame_index"].iloc[:5].apply(lambda x: x.shape)

0    (200,)
1    (200,)
2    (200,)
3    (200,)
4    (200,)
Name: trial_frame_index, dtype: object

- Getting the timestamps for each frame

In [134]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF["competitiveness_timestamp_ranges"] = cluster_exploded_TRIALS_AND_SPECTRAL_DF.apply(lambda x: update_tuples_in_list(x["competitiveness_frame_ranges"], x["video_timestamps"]), axis=1)

In [135]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF[cluster_exploded_TRIALS_AND_SPECTRAL_DF["trial_label"] != "rewarded"]["competitiveness_timestamp_ranges"]

42                                 [(34336303, 34469346)]
43                                 [(34336303, 34469346)]
44      [(35288395, 35372933), (35395107, 35422824), (...
45      [(35288395, 35372933), (35395107, 35422824), (...
46           [(36377688, 36427580), (36478857, 36543993)]
                              ...                        
1136         [(49607199, 49682036), (49769346, 49806764)]
1139                               [(52007525, 52207090)]
1140                               [(52007525, 52207090)]
1141         [(54222145, 54314998), (54346873, 54406465)]
1142         [(54222145, 54314998), (54346873, 54406465)]
Name: competitiveness_timestamp_ranges, Length: 506, dtype: object

In [136]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF["competitiveness_timestamp_ranges"]

0                               [(1276384, 1475949)]
1                               [(1276384, 1475949)]
2                               [(3276194, 3477145)]
3                               [(3276194, 3477145)]
4                               [(4476357, 4677308)]
                            ...                     
1136    [(49607199, 49682036), (49769346, 49806764)]
1139                          [(52007525, 52207090)]
1140                          [(52007525, 52207090)]
1141    [(54222145, 54314998), (54346873, 54406465)]
1142    [(54222145, 54314998), (54346873, 54406465)]
Name: competitiveness_timestamp_ranges, Length: 1010, dtype: object

- Getting all the spectral timestamps that are in the ranges

In [137]:
timestamps_col = [col for col in cluster_exploded_TRIALS_AND_SPECTRAL_DF.columns if "timestamps" in col and "video" not in col and "trial" in col]

In [111]:
timestamps_col

['trial_coherence_timestamps',
 'trial_granger_timestamps',
 'trial_power_timestamps',
 'trial_lfp_timestamps']

- Getting all the indexes that fit within each range

In [ ]:
for col in timestamps_col:
    base_col = col.replace("trial_", "").replace("timestamps", "index")
    print(base_col)
    cluster_exploded_TRIALS_AND_SPECTRAL_DF["cluster_filtered_{}".format(base_col)] = cluster_exploded_TRIALS_AND_SPECTRAL_DF.apply(lambda x: find_indices_within_ranges(x["competitiveness_timestamp_ranges"], x[col]), axis=1)

In [ ]:
base_col

In [ ]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF["trial_mPFC_theta_band"].iloc[0].shape

In [ ]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF["cluster_filtered_{}".format(base_col)].iloc[0][:10]

In [ ]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF["cluster_filtered_{}".format(base_col)].iloc[0][-10:]

In [ ]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF.head()

In [ ]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF.tail()

In [98]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF = cluster_exploded_TRIALS_AND_SPECTRAL_DF[~cluster_exploded_TRIALS_AND_SPECTRAL_DF["video_name"].apply(lambda x: ".2" in x)]

In [99]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF = cluster_exploded_TRIALS_AND_SPECTRAL_DF[~cluster_exploded_TRIALS_AND_SPECTRAL_DF["trial_and_competitiveness_label"].apply(lambda x: "temp" in x)]

In [ ]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF.groupby(["video_name", "current_subject"]).count()

# Filtering for power

In [101]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF = cluster_exploded_TRIALS_AND_SPECTRAL_DF[cluster_exploded_TRIALS_AND_SPECTRAL_DF["cluster_filtered_power_index"].apply(lambda x: len(x)) >= 1] 

In [ ]:
list(cluster_exploded_TRIALS_AND_SPECTRAL_DF.columns)

In [103]:
power_columns = [col for col in cluster_exploded_TRIALS_AND_SPECTRAL_DF if "trial" in col and ("power" in col or "granger" in col or "coherence" in col)]

In [ ]:
power_columns

- Getting the timestamps of all the clusters

In [ ]:
for col in power_columns:
    print(col)
    cluster_exploded_TRIALS_AND_SPECTRAL_DF["cluster_all_{}".format(col)] = cluster_exploded_TRIALS_AND_SPECTRAL_DF.apply(lambda x: x[col][x["cluster_filtered_power_index"]], axis=1)

In [106]:
cluster_all_columns = [col for col in cluster_exploded_TRIALS_AND_SPECTRAL_DF if "cluster_all" in col]

In [ ]:
cluster_all_columns

- Aggregating all the values within a given trial

In [ ]:
for col in cluster_all_columns:
    updated_column = col.replace("cluster_all", "cluster_mean")
    if "gamma" in col or "theta" in col:
        cluster_exploded_TRIALS_AND_SPECTRAL_DF[updated_column] = cluster_exploded_TRIALS_AND_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col]), axis=1)
    else:
        cluster_exploded_TRIALS_AND_SPECTRAL_DF[updated_column] = cluster_exploded_TRIALS_AND_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col], axis=0), axis=1)
    print(updated_column)
    print(cluster_exploded_TRIALS_AND_SPECTRAL_DF[updated_column].iloc[0])
        

In [109]:
cluster_mean_columns = [col for col in cluster_exploded_TRIALS_AND_SPECTRAL_DF if "cluster_mean" in col and "all_frequencies_all_windows" in col]

In [ ]:
cluster_mean_columns

In [ ]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF

In [ ]:
low_freq = 0
high_freq = 51
current_frequencies = range(low_freq, high_freq)

# Iterating through each brain region
for col in cluster_mean_columns:
    if "all_frequencies" not in col:
        continue
    print(col)

    fig, ax = plt.subplots()
    plt.xlim(low_freq, high_freq-1) 
    plt.xticks(np.arange(low_freq, high_freq, 5))

    stacked_df = cluster_exploded_TRIALS_AND_SPECTRAL_DF.groupby("trial_and_competitiveness_label").agg({col: stack_arrays})
    stacked_df = stacked_df.reset_index()
    stacked_df = stacked_df[~stacked_df["trial_and_competitiveness_label"].str.contains("tie")]

    stacked_df["color"] = stacked_df["trial_and_competitiveness_label"].map(comp_id_to_color)
    
    if "power" in col:
        # plt.ylim(0,0.01)
        # plt.yscale("log")
        plt.ylim((10**-3.2,10**-1.2))
        # plt.set_ylim(auto=True)
        # plt.ylim(0, 0.5)
        ax.set_yscale('log')
        # plt.ylim(0, max_value)
    elif "coherence" in col:
        plt.ylim(0.3, 0.85)
        plt.yticks(np.arange(0.3, 0.85, 0.1)) 
    elif "granger" in col:
        plt.ylim(0.2, 0.8)
        plt.yticks(np.arange(0.2, 0.85, 0.1)) 
    else:
        # pass
        plt.ylim()
    
    # Iterating through each trial type
    for index, row in stacked_df.iterrows():
        print(row["trial_and_competitiveness_label"])
        print(row[col].shape)
        #### DURING TRIAL ####
        mean_power = np.nanmean(row[col], axis=0)[low_freq: high_freq]
        
        sem_power = np.nanstd(row[col], axis=0) / np.sqrt(row[col].shape[0])
        sem_power = sem_power[low_freq: high_freq]
        print(sem_power)

        plt.fill_between(current_frequencies, 
        mean_power - sem_power, mean_power + sem_power, \
        alpha=0.3, color=row["color"])

        plt.plot(current_frequencies, mean_power, \
        label="{}".format(row["trial_and_competitiveness_label"]), linewidth=3, color=row["color"])
    
    if "granger" in col:
        title = "{}".format(col.replace("cluster_mean_trial_", "").replace("cluster_mean_trial_and_post_", "").replace("RMS_filtered_", "").replace("all_frequencies_all_windows", "").replace("power", "").replace("granger", "")).replace("coherence", "").replace("_", " ").replace("  ", " ").strip().replace(" ", " to ").strip()
    else:
        title = "{}".format(col.replace("cluster_mean_trial_", "").replace("cluster_mean_trial_and_post_", "").replace("RMS_filtered_", "").replace("all_frequencies_all_windows", "").replace("power", "").replace("granger", "")).replace("coherence", "").replace("_", " ").replace("  ", " ").strip()

    plt.title(title)
    plt.xlabel("Frequency (Hz)", fontsize=25)
    
    if "power" in col:
        # plt.ylabel("Normalized Power (a.u.)")
        output_dir = "./proc/cluster_spectra_plots/power"

    elif "coherence" in col:
        # plt.ylabel("Coherence")
        output_dir = "./proc/cluster_spectra_plots/coherence"

    elif "granger" in col:
        # plt.ylabel("Granger's Causality")
        output_dir = "./proc/cluster_spectra_plots/granger"
    
    os.makedirs(output_dir, exist_ok=True)

    # plt.legend(fontsize=10)
    
    # Hide top and right borders
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # Leave bottom and left spines
    ax.spines['bottom'].set_visible(True)
    ax.spines['left'].set_visible(True)
    ax.spines['bottom'].set_linewidth(4)
    ax.spines['left'].set_linewidth(4)

    ax.tick_params(length=8, width=4)
    plt.yticks(fontsize=25)
    plt.xticks(fontsize=25)

    plt.tight_layout()    
    plt.savefig(os.path.join(output_dir, "cluster_{}.png".format(col.replace("cluster_mean_trial_and_post_", "").replace("RMS_filtered_", "").replace("all_frequencies_all_windows", "").strip("_").strip())), transparent=True)
    plt.show()


In [113]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF.to_pickle("./proc/cluster_exploded_TRIALS_AND_SPECTRAL_DF.pkl")

In [114]:
export_cluster_exploded_TRIALS_AND_SPECTRAL_DF = cluster_exploded_TRIALS_AND_SPECTRAL_DF.copy()

In [ ]:
export_cluster_exploded_TRIALS_AND_SPECTRAL_DF.columns

In [ ]:
export_cluster_exploded_TRIALS_AND_SPECTRAL_DF

In [ ]:
raise ValueError()

## Looking at cluster related stuff

- Parsing velocity during relavent cluster

In [ ]:
for col in export_cluster_exploded_TRIALS_AND_SPECTRAL_DF.columns:
    print(col)

In [ ]:
export_cluster_exploded_TRIALS_AND_SPECTRAL_DF["cluster_mean_trial_BLA-to-LH_granger_gamma"]

In [ ]:
raise ValueError()

- Code for verifying spectram

In [ ]:
# low_freq = 0
# high_freq = 51
# current_frequencies = range(low_freq, high_freq)

# # Iterating through each brain region
# for col in cluster_mean_columns:
#     if "all_frequencies" not in col:
#         continue
#     print(col)

#     fig, ax = plt.subplots()
#     plt.xlim(low_freq, high_freq-1) 
#     plt.xticks(np.arange(low_freq, high_freq-1, 5))

#     stacked_df = cluster_exploded_TRIALS_AND_SPECTRAL_DF.groupby("trial_and_competitiveness_label").agg({col: stack_arrays})
#     stacked_df = stacked_df.reset_index()
#     stacked_df = stacked_df[~stacked_df["trial_and_competitiveness_label"].str.contains("tie")]

#     stacked_df["color"] = stacked_df["trial_and_competitiveness_label"].map(comp_id_to_color)
    
#     if "power" in col:
#         # plt.ylim(0,0.01)
#         # plt.yscale("log")
#         # plt.ylim((10**-3.5,10**-0.5))
#         # plt.set_ylim(auto=True)
#         plt.ylim()
#         ax.set_yscale('log')
#         # plt.ylim(0, max_value)
        
#     else:
#         # pass
#         plt.ylim(0, 0.8)
    
#     # Iterating through each trial type
#     counter = 0
#     for index, row in stacked_df.iloc[:100].iterrows():
#         print(row["trial_and_competitiveness_label"])
#         print(row[col].shape)
#         for trace in row[col]: 
#             plt.plot(current_frequencies, trace[low_freq: high_freq], color=row["color"], alpha=0.5)
#             counter += 1
#             if counter >= 100:
#                 break
    
#     if "granger" in col:
#         title = "{}".format(col.replace("cluster_mean_trial_", "").replace("cluster_mean_trial_and_post_", "").replace("RMS_filtered_", "").replace("all_frequencies_all_windows", "").replace("power", "").replace("granger", "")).replace("coherence", "").replace("_", " ").replace("  ", " ").strip().replace(" ", " to ").strip()
#     else:
#         title = "{}".format(col.replace("cluster_mean_trial_", "").replace("cluster_mean_trial_and_post_", "").replace("RMS_filtered_", "").replace("all_frequencies_all_windows", "").replace("power", "").replace("granger", "")).replace("coherence", "").replace("_", " ").replace("  ", " ").strip()

#     plt.title(title)
#     plt.xlabel("Frequency (Hz)")
    
#     if "power" in col:
#         # plt.ylabel("Normalized Power (a.u.)")
#         output_dir = "./proc/cluster_spectra_plots/power"

#     elif "coherence" in col:
#         # plt.ylabel("Coherence")
#         output_dir = "./proc/cluster_spectra_plots/coherence"

#     elif "granger" in col:
#         # plt.ylabel("Granger's Causality")
#         output_dir = "./proc/cluster_spectra_plots/granger"
    
#     os.makedirs(output_dir, exist_ok=True)

#     # plt.legend(fontsize=10)

#     # Hide top and right borders
#     ax.spines['top'].set_visible(False)
#     ax.spines['right'].set_visible(False)

#     # Leave bottom and left spines
#     ax.spines['bottom'].set_visible(True)
#     ax.spines['left'].set_visible(True)
#     ax.spines['bottom'].set_linewidth(2)
#     ax.spines['left'].set_linewidth(2)

#     plt.tight_layout()    
#     plt.show()
#     break

In [101]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF.to_pickle("./proc/cluster_exploded_TRIALS_AND_SPECTRAL_DF.pkl")

# Plotting averages

In [102]:
GROUPING = "trial_and_competitiveness_label"

In [103]:
theta_gamma_mean_columns = [col for col in cluster_exploded_TRIALS_AND_SPECTRAL_DF if "cluster_mean" in col and "all_frequencies_all_windows" not in col and ("gamma" in col or "theta" in col)]

In [ ]:
theta_gamma_mean_columns

In [ ]:
col

In [107]:
# for col in theta_gamma_mean_columns:
#     fig, ax = plt.subplots()
#     mean_df = pd.DataFrame(cluster_exploded_TRIALS_AND_SPECTRAL_DF[[GROUPING, col]].groupby(GROUPING).mean()[col]).rename(columns={col: "mean"})
#     sem_df = pd.DataFrame(cluster_exploded_TRIALS_AND_SPECTRAL_DF[[GROUPING, col]].groupby(GROUPING).agg("sem")[col]).rename(columns={col: "sem"})
#     merged_df = pd.merge(mean_df, sem_df, on=GROUPING).reset_index().sort_values([GROUPING])
#     merged_df["measurement"] = col
#     for index, row in merged_df.iterrows():
#         if "tie" in row[GROUPING]:
#             continue
#         plt.bar("\n".join(row[GROUPING].split("_")).replace("competitive", "comp"), row["mean"], color=comp_id_to_color[row[GROUPING]])
#         plt.errorbar("\n".join(row[GROUPING].split("_")).replace("competitive", "comp"), row["mean"], yerr=row["sem"], capsize=20, ecolor = "black")

#     ax.tick_params(axis='x', rotation=90)
#     brain_region = col.replace("cluster_mean_trial_", "").replace("cluster_mean_trial_and_post", "").replace("coherence", "").replace("granger", "").replace("power", "").replace("__", "_").replace("_", " ").strip()
#     plt.title(brain_region.replace("gamma", "").replace("theta", "").strip())
    
#     if "power" in col:
#         plt.ylim()
#     else:
#         plt.ylim(0, 0.8)


#     plt.tight_layout()
#     # plt.savefig("./proc/cluster_average_spectra/{}_averages.png".format(col.replace("cluster_mean_trial_", "")))
#     plt.close()
#     # plt.show()
#     # for group in cluster_exploded_TRIALS_AND_SPECTRAL_DF[GROUPING].unique():
#     #     current_df = cluster_exploded_TRIALS_AND_SPECTRAL_DF[cluster_exploded_TRIALS_AND_SPECTRAL_DF[GROUPING] == group]
    

In [ ]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF["trial_and_competitiveness_label"].unique()

In [112]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF[["trial_and_competitiveness_label", "current_subject", "recording", "video_name"] + theta_gamma_mean_columns].to_excel("./proc/competitiveness_trials_and_spectral_mean.xlsx")

In [113]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF[["trial_and_competitiveness_label", "current_subject", "recording", "video_name"] + theta_gamma_mean_columns].to_csv("./proc/competitiveness_trials_and_spectral_mean.csv")

# Plotting power bar graphs

In [116]:
    x_pos = np.arange(0,1,len(merged_df))

In [ ]:
x_pos

In [180]:
for col in theta_gamma_mean_columns:
    if "power" not in col:
        continue

    fig, ax = plt.subplots(figsize=(8,4))
    # plt.figure(figsize=(10,6))

    mean_df = pd.DataFrame(cluster_exploded_TRIALS_AND_SPECTRAL_DF[[GROUPING, col]].groupby(GROUPING).mean()[col]).rename(columns={col: "mean"})
    sem_df = pd.DataFrame(cluster_exploded_TRIALS_AND_SPECTRAL_DF[[GROUPING, col]].groupby(GROUPING).agg("sem")[col]).rename(columns={col: "sem"})
    merged_df = pd.merge(mean_df, sem_df, on=GROUPING).reset_index().sort_values([GROUPING])
    merged_df["measurement"] = col
    x_pos = np.linspace(0,0.5,len(merged_df)) * 0.9
    
    for index, row in merged_df.sort_values(by=["trial_and_competitiveness_label"]).iloc[[3,4,2,0,1]].reset_index().iterrows():
        
        plt.bar(x_pos[index], row["mean"], color=comp_id_to_color[row[GROUPING]], width=0.5/len(merged_df))
        plt.errorbar(x_pos[index], row["mean"], yerr=row["sem"], capsize=20, ecolor = "black", elinewidth=3, markeredgewidth=3)

        # plt.bar("\n".join(row[GROUPING].split("_")).replace("competitive", "comp"), row["mean"], color=comp_id_to_color[row[GROUPING]], width=0.5)
        # plt.errorbar("\n".join(row[GROUPING].split("_")).replace("competitive", "comp"), row["mean"], yerr=row["sem"], capsize=20, ecolor = "black")

    plt.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom=False,      # ticks along the bottom edge are off
        top=False,         # ticks along the top edge are off
        labelbottom=False) # labels along the bottom edge are off

    ax.tick_params(axis='x', rotation=90)
    brain_region = col.replace("cluster_mean_trial_", "").replace("cluster_mean_trial_and_post", "").replace("coherence", "").replace("granger", "").replace("power", "").replace("__", "_").replace("_", " ").strip()
    # plt.title(brain_region.replace("gamma", "").replace("theta", "").strip())
    
    if "theta" in col:
        plt.ylim(0, 0.04)
    elif "gamma" in col:
        plt.ylim(0, 0.004)
    else:
        plt.ylim(0, 0.8)
    ax.tick_params(length=10, width=4)
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    plt.yticks(fontsize=30)

    # Hide top and right borders
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # Leave bottom and left spines
    ax.spines['bottom'].set_visible(True)
    ax.spines['left'].set_visible(True)
    ax.spines['bottom'].set_linewidth(4)
    ax.spines['left'].set_linewidth(4)

    plt.tight_layout()
    plt.savefig("./proc/cluster_average_spectra/power/{}_averages.png".format(col.replace("cluster_mean_trial_", "")), transparent=True)
    plt.close()
    # plt.show()
    # for group in cluster_exploded_TRIALS_AND_SPECTRAL_DF[GROUPING].unique():
    #     current_df = cluster_exploded_TRIALS_AND_SPECTRAL_DF[cluster_exploded_TRIALS_AND_SPECTRAL_DF[GROUPING] == group]
    

# Plotting coherence plots

In [236]:
for col in theta_gamma_mean_columns:
    if "coherence" not in col:
        continue


    fig, ax = plt.subplots(figsize=(8,4))
    # plt.figure(figsize=(10,6))

    mean_df = pd.DataFrame(cluster_exploded_TRIALS_AND_SPECTRAL_DF[[GROUPING, col]].groupby(GROUPING).mean()[col]).rename(columns={col: "mean"})
    sem_df = pd.DataFrame(cluster_exploded_TRIALS_AND_SPECTRAL_DF[[GROUPING, col]].groupby(GROUPING).agg("sem")[col]).rename(columns={col: "sem"})
    merged_df = pd.merge(mean_df, sem_df, on=GROUPING).reset_index().sort_values([GROUPING])
    merged_df["measurement"] = col
    x_pos = np.linspace(0,0.5,len(merged_df)) * 0.9
    
    for index, row in merged_df.sort_values(by=["trial_and_competitiveness_label"]).iloc[[3,4,2,0,1]].reset_index().iterrows():
        
        plt.bar(x_pos[index], row["mean"], color=comp_id_to_color[row[GROUPING]], width=0.5/len(merged_df))
        plt.errorbar(x_pos[index], row["mean"], yerr=row["sem"], capsize=20, ecolor = "black", elinewidth=3, markeredgewidth=3)

        # plt.bar("\n".join(row[GROUPING].split("_")).replace("competitive", "comp"), row["mean"], color=comp_id_to_color[row[GROUPING]], width=0.5)
        # plt.errorbar("\n".join(row[GROUPING].split("_")).replace("competitive", "comp"), row["mean"], yerr=row["sem"], capsize=20, ecolor = "black")

    plt.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom=False,      # ticks along the bottom edge are off
        top=False,         # ticks along the top edge are off
        labelbottom=False) # labels along the bottom edge are off

    ax.tick_params(axis='x', rotation=90)
    brain_region = col.replace("cluster_mean_trial_", "").replace("cluster_mean_trial_and_post", "").replace("coherence", "").replace("granger", "").replace("power", "").replace("__", "_").replace("_", " ").strip()
    # plt.title(brain_region.replace("gamma", "").replace("theta", "").strip())
    
    if "theta" in col:
        plt.ylim(0, 0.8)
    elif "gamma" in col:
        plt.ylim(0, 0.8)
    else:
        plt.ylim(0, 0.8)
    ax.tick_params(length=10, width=4)
    plt.yticks(fontsize=30)

    # Hide top and right borders
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # Leave bottom and left spines
    ax.spines['bottom'].set_visible(True)
    ax.spines['left'].set_visible(True)
    ax.spines['bottom'].set_linewidth(4)
    ax.spines['left'].set_linewidth(4)

    plt.tight_layout()
    plt.savefig("./proc/cluster_average_spectra/coherence/{}_averages.png".format(col.replace("cluster_mean_trial_", "")), transparent=True)
    plt.close()
    # plt.show()
    # for group in cluster_exploded_TRIALS_AND_SPECTRAL_DF[GROUPING].unique():
    #     current_df = cluster_exploded_TRIALS_AND_SPECTRAL_DF[cluster_exploded_TRIALS_AND_SPECTRAL_DF[GROUPING] == group]
    

# Plotting Granger averages

In [235]:
for col in theta_gamma_mean_columns:
    if "granger" not in col:
        continue


    fig, ax = plt.subplots(figsize=(8,4))
    # plt.figure(figsize=(10,6))

    mean_df = pd.DataFrame(cluster_exploded_TRIALS_AND_SPECTRAL_DF[[GROUPING, col]].groupby(GROUPING).mean()[col]).rename(columns={col: "mean"})
    sem_df = pd.DataFrame(cluster_exploded_TRIALS_AND_SPECTRAL_DF[[GROUPING, col]].groupby(GROUPING).agg("sem")[col]).rename(columns={col: "sem"})
    merged_df = pd.merge(mean_df, sem_df, on=GROUPING).reset_index().sort_values([GROUPING])
    merged_df["measurement"] = col
    x_pos = np.linspace(0,0.5,len(merged_df)) * 0.9
    
    for index, row in merged_df.sort_values(by=["trial_and_competitiveness_label"]).iloc[[3,4,2,0,1]].reset_index().iterrows():
        
        plt.bar(x_pos[index], row["mean"], color=comp_id_to_color[row[GROUPING]], width=0.5/len(merged_df))
        plt.errorbar(x_pos[index], row["mean"], yerr=row["sem"], capsize=20, ecolor = "black", elinewidth=3, markeredgewidth=3)

        # plt.bar("\n".join(row[GROUPING].split("_")).replace("competitive", "comp"), row["mean"], color=comp_id_to_color[row[GROUPING]], width=0.5)
        # plt.errorbar("\n".join(row[GROUPING].split("_")).replace("competitive", "comp"), row["mean"], yerr=row["sem"], capsize=20, ecolor = "black")

    plt.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom=False,      # ticks along the bottom edge are off
        top=False,         # ticks along the top edge are off
        labelbottom=False) # labels along the bottom edge are off

    ax.tick_params(axis='x', rotation=90)
    brain_region = col.replace("cluster_mean_trial_", "").replace("cluster_mean_trial_and_post", "").replace("coherence", "").replace("granger", "").replace("power", "").replace("__", "_").replace("_", " ").strip()
    # plt.title(brain_region.replace("gamma", "").replace("theta", "").strip())
    
    if "theta" in col:
        plt.ylim(0, 0.8)
    elif "gamma" in col:
        plt.ylim(0, 0.8)
    else:
        plt.ylim(0, 0.8)
    ax.tick_params(length=10, width=4)
    plt.yticks(fontsize=30)

    # Hide top and right borders
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # Leave bottom and left spines
    ax.spines['bottom'].set_visible(True)
    ax.spines['left'].set_visible(True)
    ax.spines['bottom'].set_linewidth(4)
    ax.spines['left'].set_linewidth(4)

    plt.tight_layout()
    plt.savefig("./proc/cluster_average_spectra/granger/{}_averages.png".format(col.replace("cluster_mean_trial_", "")), transparent=True)
    plt.close()
    # plt.show()
    # for group in cluster_exploded_TRIALS_AND_SPECTRAL_DF[GROUPING].unique():
    #     current_df = cluster_exploded_TRIALS_AND_SPECTRAL_DF[cluster_exploded_TRIALS_AND_SPECTRAL_DF[GROUPING] == group]
    

In [ ]:
merged_df

# Checking significance

In [ ]:
all_significance_csv = []
for file_path in glob.glob("./significance_output/emmeans_coherence/*csv"):
    current_df = pd.read_csv(file_path, index_col=0)
    file_name_parts = os.path.basename(file_path).replace(".csv", "").replace("cluster_mean_trial_", "").replace("_emmeans", "").replace("_coherence_", " ").split(" ")
    
    print(file_name_parts)
    current_df["brain_region"] = file_name_parts[0]
    current_df["band"] = file_name_parts[1]

    all_significance_csv.append(current_df)

In [196]:
all_significance_df = pd.concat(all_significance_csv)

In [198]:
all_significance_df["p.value"] = all_significance_df["p.value"].astype(float)

In [ ]:
all_significance_df["contrast"].unique()

In [204]:
good_contrasts = ['rewarded - win_competitive', 'rewarded - win_no_comp',
       'win_competitive - win_no_comp',
       'win_competitive - lose_competitive',
       'win_no_comp - lose_no_comp', 'lose_competitive - lose_no_comp']

In [ ]:
all_significance_df[(all_significance_df["p.value"] <= 0.001) & (all_significance_df["contrast"].isin(good_contrasts)) & (~all_significance_df["brain_region"].str.contains("mPFC"))].sort_values(by=["band", "brain_region", "contrast"])

In [ ]:
all_significance_csv[0]

# OLD Stuff

In [ ]:
granger_col = [col for col in cluster_exploded_TRIALS_AND_SPECTRAL_DF.columns if ("theta" in col or "gamma" in col) and "granger" in col and "mean" in col]

In [ ]:
granger_col

In [ ]:
for col in granger_col:
    fig, ax = plt.subplots()
    mean_df = pd.DataFrame(cluster_exploded_TRIALS_AND_SPECTRAL_DF[[GROUPING, col]].groupby(GROUPING).mean()[col]).rename(columns={col: "mean"})
    sem_df = pd.DataFrame(cluster_exploded_TRIALS_AND_SPECTRAL_DF[[GROUPING, col]].groupby(GROUPING).agg("sem")[col]).rename(columns={col: "sem"})
    merged_df = pd.merge(mean_df, sem_df, on=GROUPING).reset_index().sort_values([GROUPING])
    merged_df["measurement"] = col
    for index, row in merged_df.iterrows():
        plt.bar("\n".join(row[GROUPING].split("_")).replace("competitive", "comp"), row["mean"], color=comp_id_to_color[row[GROUPING]])
        plt.errorbar("\n".join(row[GROUPING].split("_")).replace("competitive", "comp"), row["mean"], yerr=row["sem"], capsize=20, ecolor = "black")

    ax.tick_params(axis='x', rotation=90)
    brain_region = col.replace("cluster_mean_trial_and_post", "").replace("coherence", "").replace("__", "_").replace("_", " ").strip()
    plt.title(brain_region.replace("gamma", "").replace("theta", "").strip())
    plt.ylim(0, 0.6)
    plt.tight_layout()
    plt.savefig("./proc/cluster_spectra_plots/mean_granger/{} granger.png".format(brain_region))
    # for group in cluster_exploded_TRIALS_AND_SPECTRAL_DF[GROUPING].unique():
    #     current_df = cluster_exploded_TRIALS_AND_SPECTRAL_DF[cluster_exploded_TRIALS_AND_SPECTRAL_DF[GROUPING] == group]
    

In [ ]:
raise ValueError()

# LDA

In [ ]:
power_columns = [col for col in TRIALS_AND_SPECTRAL_DF if ("gamma" in col or "theta" in col) and ("trial_and_post" in col or "cluster" in col) and "phase" not in col and "band" not in col or "cluster_filtered" in col] 

In [ ]:
power_columns

- Making a separate row for each cluster

In [ ]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF[to_keep_columns + power_columns].explode(["cluster_filtered_power_timestamps"])

In [ ]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF["cluster_filtered_granger_timestamps"]

- Making independent columns for cluster name and indexes

In [ ]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF["power_cluster_id"] = cluster_exploded_TRIALS_AND_SPECTRAL_DF["cluster_filtered_power_timestamps"].apply(lambda x: x[0])
cluster_exploded_TRIALS_AND_SPECTRAL_DF["power_cluster_indexes"] = cluster_exploded_TRIALS_AND_SPECTRAL_DF["cluster_filtered_power_timestamps"].apply(lambda x: x[1])

In [ ]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF

In [ ]:
power_columns = [col for col in cluster_exploded_TRIALS_AND_SPECTRAL_DF if "granger" in col and "trial_and_post" in col and "timestamps" not in col]


power_columns = [col for col in TRIALS_AND_SPECTRAL_DF if ("gamma" in col or "theta" in col) and ("trial_and_post" in col or "cluster" in col) and "phase" not in col and "band" not in col] 

In [ ]:
power_columns

- Getting the timestamps of all the clusters

In [ ]:
for col in power_columns:
    print(col)
    cluster_exploded_TRIALS_AND_SPECTRAL_DF["cluster_all_{}".format(col)] = cluster_exploded_TRIALS_AND_SPECTRAL_DF.apply(lambda x: x[col][x["power_cluster_indexes"]], axis=1)

In [ ]:
cluster_all_columns = [col for col in cluster_exploded_TRIALS_AND_SPECTRAL_DF if "cluster_all" in col]

In [ ]:
cluster_all_columns

- Aggregating all the values within a given trial

In [ ]:
for col in cluster_all_columns:
    updated_column = col.replace("cluster_all", "cluster_mean")
    if "gamma" in col or "theta" in col:
        cluster_exploded_TRIALS_AND_SPECTRAL_DF[updated_column] = cluster_exploded_TRIALS_AND_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col]), axis=1)
    else:
        cluster_exploded_TRIALS_AND_SPECTRAL_DF[updated_column] = cluster_exploded_TRIALS_AND_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col], axis=0), axis=1)
    print(updated_column)
    print(cluster_exploded_TRIALS_AND_SPECTRAL_DF[updated_column].iloc[0])
        

In [ ]:
cluster_mean_columns = [col for col in cluster_exploded_TRIALS_AND_SPECTRAL_DF if "cluster_mean" in col]

In [ ]:
cluster_mean_columns = [col for col in cluster_mean_columns if "mPFC" in col and "power" not in col and "coherence" in col]

In [ ]:
cluster_mean_columns

In [ ]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF = cluster_exploded_TRIALS_AND_SPECTRAL_DF[~cluster_exploded_TRIALS_AND_SPECTRAL_DF["power_cluster_id"].str.contains("tie")]

In [ ]:
# cluster_exploded_TRIALS_AND_SPECTRAL_DF = cluster_exploded_TRIALS_AND_SPECTRAL_DF[cluster_exploded_TRIALS_AND_SPECTRAL_DF["power_cluster_id"].isin(["lose_no_comp", "lose_competitive"])].reset_index(drop=True)

In [ ]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF["power_cluster_id"]

In [ ]:
# GROUPING = "competition_closeness"


GROUPING = "trial_label"	
GROUPING = "competition_closeness"
GROUPING = "power_cluster_id"





In [ ]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF["factorized_{}".format(GROUPING)], unique = pd.factorize(cluster_exploded_TRIALS_AND_SPECTRAL_DF[GROUPING])


In [ ]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF = cluster_exploded_TRIALS_AND_SPECTRAL_DF.dropna(subset=cluster_mean_columns)

In [ ]:
network_array = cluster_exploded_TRIALS_AND_SPECTRAL_DF[cluster_mean_columns].values

In [ ]:
cluster_exploded_TRIALS_AND_SPECTRAL_DF[GROUPING].unique()

# Reducing Dimensions

In [ ]:
network_array

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
from umap import UMAP


In [ ]:
scaler = StandardScaler()
reduced_dimension_network_array = scaler.fit_transform(network_array)





In [ ]:
# lda = LinearDiscriminantAnalysis()
# lda = LinearDiscriminantAnalysis(n_components=2)
# reduced_dimension_network_array = lda.fit_transform(reduced_dimension_network_array, cluster_exploded_TRIALS_AND_SPECTRAL_DF[GROUPING])
# reduced_dimension_network_array = lda.fit_transform(reduced_dimension_network_array)

In [ ]:
# lda = PCA(n_components=2)
# reduced_dimension_network_array = lda.fit_transform(reduced_dimension_network_array)

In [ ]:
reduced_dimension_network_array = UMAP(
    n_components=2,
    random_state=42,
).fit_transform(network_array, y=cluster_exploded_TRIALS_AND_SPECTRAL_DF["factorized_{}".format(GROUPING)].values)


In [ ]:
reduced_dimension_network_array.shape

In [ ]:
# reduced_dimension_network_array = lda.fit_transform(reduced_dimension_network_array, cluster_exploded_TRIALS_AND_SPECTRAL_DF[GROUPING])


In [ ]:
reduced_dimension_network_array.shape

In [ ]:
GROUPING = "current_subject"
# GROUPING = "video_name"
# GROUPING = "power_cluster_id"


In [ ]:
reduced_dimension_network_array

In [ ]:
plt.figure(figsize=(5,5))
for group in cluster_exploded_TRIALS_AND_SPECTRAL_DF[GROUPING].unique():
    current_df = cluster_exploded_TRIALS_AND_SPECTRAL_DF[cluster_exploded_TRIALS_AND_SPECTRAL_DF[GROUPING] == group]
    print(current_df.index)
    plot = plt.scatter(reduced_dimension_network_array[current_df.index,0], reduced_dimension_network_array[current_df.index,1], label=group, alpha=0.8)#, c=comp_id_to_color[group], )
    
plt.title("LDA of LFP features", fontsize=20)
plt.xlabel("LD1", fontsize=20)
plt.ylabel("LD2", fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
# plt.legend()
plt.tight_layout()

os.makedirs("./proc/network", exist_ok=True)

plt.savefig("./proc/network/rf_LDA_outcome_labeled.png")
plt.savefig("./proc/network/rf_LDA_outcome_labeled.eps")
plt.show()

In [ ]:
plt.figure(figsize=(5,5))
for group in cluster_exploded_TRIALS_AND_SPECTRAL_DF[GROUPING].unique():
    current_df = cluster_exploded_TRIALS_AND_SPECTRAL_DF[cluster_exploded_TRIALS_AND_SPECTRAL_DF[GROUPING] == group]
    print(current_df.index)
    plot = plt.hist(reduced_dimension_network_array[current_df.index,0], color=comp_id_to_color[group], label=group, alpha=0.8)
    
plt.title("LDA of LFP features", fontsize=20)
plt.xlabel("LD1", fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
# plt.legend()
plt.tight_layout()

os.makedirs("./proc/network", exist_ok=True)

plt.savefig("./proc/network/rf_LDA_outcome_labeled.png")
plt.savefig("./proc/network/rf_LDA_outcome_labeled.eps")
plt.show()

In [ ]:
raise ValueError()

In [ ]:
plt.figure(figsize=(5,5))
for group in cluster_exploded_TRIALS_AND_SPECTRAL_DF["current_subject"].unique():
    current_df = cluster_exploded_TRIALS_AND_SPECTRAL_DF[cluster_exploded_TRIALS_AND_SPECTRAL_DF["current_subject"] == group]
    
    plot = plt.scatter(reduced_dimension_network_array[current_df.index,0], reduced_dimension_network_array[current_df.index,1], label=group)

    
plt.title("LDA of LFP features", fontsize=20)
plt.xlabel("LD1", fontsize=20)
plt.ylabel("LD2", fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.legend()
plt.tight_layout()
plt.savefig("./proc/network/rf_LDA_subject_labeled.png")
plt.savefig("./proc/network/rf_LDA_subject_labeled.eps")
plt.show()

In [ ]:
# Get the coefficients (loadings) of each feature
loadings = lda.scalings_

# Calculate importance for each feature for each discriminant
# The importance is calculated as the square of each coefficient
importance = np.square(loadings)



In [ ]:
lda_importance = pd.DataFrame(importance)
lda_importance["feature"] = feature_columns

In [ ]:
important_features = []

In [ ]:
lda_importance.sort_values(0, ascending=False)

In [ ]:
important_features += lda_importance.sort_values(0, ascending=False).head(n=4)["feature"].to_list()

In [ ]:
important_features += lda_importance.sort_values(1, ascending=False).head(n=4)["feature"].to_list()

In [ ]:
filtered_lda_importance = lda_importance[lda_importance["feature"].isin(important_features)]

In [ ]:
filtered_lda_importance = filtered_lda_importance.sort_values(0, ascending=False)

In [ ]:
filtered_lda_importance["feature"] = filtered_lda_importance["feature"].apply(lambda x: x.replace("_band_", "\n").replace("_", " "))

In [ ]:
# Sample data
list1 = filtered_lda_importance[0]
list2 = filtered_lda_importance[1]
indices = range(len(list1))

# Creating the plot
fig, ax1 = plt.subplots(figsize=(6,6))
    # fig, ax = plt.subplots(figsize=(6,6))


# Plotting the first list
color1 = '#15616F'
# FFAF00; teal #15616F
ax1.bar(indices, list1, width=0.4, align='center', color=color1)
# ax1.set_xlabel('Index')
ax1.set_ylabel('LD1 Scalings', color=color1, fontsize=20)
ax1.tick_params(axis='y', labelcolor=color1, rotation=90)

# Creating a second Y-axis for the second list
ax2 = ax1.twinx()  
color2 = 'tab:red'
ax2.bar([i + 0.4 for i in indices], list2, width=0.4, align='center', color=color2)
ax2.set_ylabel('LD2 Scalings', color=color2, fontsize=20)
ax2.tick_params(axis='y', labelcolor=color2, rotation=90)

# Adjusting the X-axis to show labels correctly
ax1.set_xticks(indices, filtered_lda_importance['feature'].values, rotation = 90, fontsize=15)
# xticks(indices, filtered_lda_importance['feature'].values, rotation = 90)
# plt.xticks(labels=filtered_lda_importance['feature'])

plt.title('LDA feature importance', fontsize=20)
plt.tight_layout()
plt.savefig("./proc/network/lda_feature_importance.png")
plt.savefig("./proc/network/lda_feature_importance.eps")

plt.show()

In [ ]:
raise ValueError()

- Grouping all the rows with the same video and subject together

In [ ]:
list(TRIALS_AND_SPECTRAL_DF.columns)

In [ ]:
explode_columns

In [ ]:
other_explode_columns = ["tone_stop_frame",
                         "condition",
                         "competition_closeness",
                         "notes",
                         "10s_before_tone_frame",
                         "10s_after_tone_frame",
                        'cluster_index_ranges_dict',
 'cluster_times',
 'cluster_times_ranges_dict',
 'cluster_timestamps_ranges_dict',
 'trial_cluster_times_ranges_dict',
 'trial_cluster_timestamps_ranges_dict']

In [ ]:
filter_columns

In [ ]:
# Define columns to be transformed into numpy arrays


# Define aggregation dictionary
agg_dict = {col: list for col in explode_columns + other_explode_columns if col not in groupby_columns and col != "tone_start_frame"}

agg_dict.update({col: 'first' for col in filter_columns + ["tone_start_frame"] if col not in groupby_columns and col not in other_explode_columns})

# Apply groupby and aggregation
video_TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.groupby(["video_name", "current_subject"]).agg(agg_dict).reset_index()


In [ ]:
video_TRIALS_AND_SPECTRAL_DF.columns

In [ ]:
video_TRIALS_AND_SPECTRAL_DF.head()

- Combining all the dictionaries together

In [ ]:
for col in [_ for _ in video_TRIALS_AND_SPECTRAL_DF if "dict" in _]:
    video_TRIALS_AND_SPECTRAL_DF[col] = video_TRIALS_AND_SPECTRAL_DF[col].apply(lambda x: combine_dicts(x))

In [ ]:
video_TRIALS_AND_SPECTRAL_DF.head()

In [ ]:
video_TRIALS_AND_SPECTRAL_DF.to_pickle("./proc/{}_cluster_ranges.pkl".format(OUTPUT_PREFIX))

# Filtering out the SLEAP posed for during trials

In [ ]:
raise ValueError()

In [ ]:
TRIALS_AND_SPECTRAL_DF

In [ ]:
TRIALS_AND_SPECTRAL_DF["video_frame"] = TRIALS_AND_SPECTRAL_DF["video_timestamps"].apply(lambda x: np.array(list(range(len(x)))) + 1)

In [ ]:
TRIALS_AND_SPECTRAL_DF["video_frame"].head().apply(lambda x: x.shape)

In [ ]:
['subject_thorax_to_agent_thorax',
 'nose_to_reward_port_sum',
 'nose_to_reward_port_diff',
 'thorax_velocity_sum',
 'thorax_velocity_diff',
 'to_reward_port_angle_sum',
 'to_reward_port_angle_diff',
 'subject_nose_to_reward_port',
 'subject_thorax_velocity',
 'subject_to_reward_port_angle',
 'agent_nose_to_reward_port',
 'agent_thorax_velocity',
 'agent_to_reward_port_angle',
 'closebool_subject_nose_to_reward_port',
 'closebool_agent_nose_to_reward_port',
 'movingbool_subject_thorax_velocity',
 'movingbool_agent_thorax_velocity',
 'manual_cluster_id',
 'standard_embedding_x',
 'standard_embedding_y',
 'kmeans_cluster',
 'subject_locations',
 'agent_locations',
 'subject_thorax',
 'subject_nose',
 'subject_tail_base',
 'agent_thorax',
 'agent_nose',
 'agent_tail_base']

In [ ]:
# sleap_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "locations" in col or "velocity" in col or "to_reward_port" in col or "video_frame" in col]

In [ ]:
sleap_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if ("thorax" in col or "nose" in col or "reward_port" in col or "standard_embedding" in col or "cluster" in col or "frame_index" in col or "locations" in col or "tailbase" in col) and "timestamp" not in col]


In [ ]:
sleap_columns

In [ ]:
for col in sleap_columns:
    print(col)
    print(TRIALS_AND_SPECTRAL_DF[col].apply(lambda x: x.shape).iloc[0])

In [ ]:
for col in sorted(sleap_columns):
    updated_item_col = "trial_and_post_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "trial_and_post_video_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["cluster_timestamp"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["cluster_timestamp"], items=x[col])[0], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF.head()

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_and_post_frame_index"].iloc[0].shape

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=sleap_columns + ["cluster_timestamp"], errors="ignore")

In [ ]:
for col in TRIALS_AND_SPECTRAL_DF.columns:
    print(col)

- Filtering coherence

In [ ]:
coherence_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "coherence" in col and "timestamps" not in col and "calculation" not in col]

In [ ]:
coherence_columns

In [ ]:
for col in sorted(coherence_columns):
    updated_item_col = "trial_and_post_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "trial_and_post_coherence_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[0], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=coherence_columns + ["coherence_timestamps"], errors="ignore")

- Filtering Grangers

In [ ]:
granger_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "granger" in col and "timestamps" not in col and "calculation" not in col]

In [ ]:
granger_columns

In [ ]:
for col in sorted(granger_columns):
    updated_item_col = "trial_and_post_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "trial_and_post_granger_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[0], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=granger_columns + ["granger_timestamps"], errors="ignore")

In [ ]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

- Filtering power

In [ ]:
TRIALS_AND_SPECTRAL_DF.head()

In [ ]:
power_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "power" in col and "timestamps" not in col and "calculation" not in col]

In [ ]:
power_columns

In [ ]:
for col in sorted(power_columns):
    updated_item_col = "trial_and_post_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "trial_and_post_power_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["power_timestamps"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["power_timestamps"], items=x[col])[0], axis=1)


In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=power_columns + ["power_timestamps"], errors="ignore")

In [ ]:
for col in TRIALS_AND_SPECTRAL_DF:
    print(col)

# Filtering out phase

In [ ]:
lfp_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "trace" in col and "timestamps" not in col and "calculation" not in col]

In [ ]:
band_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "band" in col and "timestamps" not in col and "calculation" not in col]

In [ ]:
phase_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "phase" in col and "timestamps" not in col and "calculation" not in col]

In [ ]:
phase_columns = phase_columns + band_columns +lfp_columns

In [ ]:
phase_columns

In [ ]:
for col in sorted(phase_columns):
    updated_item_col = "trial_and_post_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "trial_and_post_lfp_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["lfp_timestamps"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["lfp_timestamps"], items=x[col])[0], axis=1)


In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_and_post_lfp_timestamps"]

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=phase_columns + ["lfp_timestamps"], errors="ignore")

In [ ]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [ ]:
for col in TRIALS_AND_SPECTRAL_DF:
    print(col)

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_and_post_kmeans_cluster"].iloc[0].shape

In [ ]:
raise ValueError()

# Filtering for rows that are in the video

In [ ]:
TRIALS_AND_SPECTRAL_DF["in_video"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: x["start_frame"] <= x["trial_video_frame"][0] <= x["stop_frame"], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["in_video"]].reset_index()

In [ ]:
TRIALS_AND_SPECTRAL_DF

In [ ]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [ ]:
raise ValueError()

# Filtering out spikes

In [ ]:
raise ValueError()

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_spike_times"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: filter_spike_times(x["spike_times"], start=x["tone_start_timestamp"], stop=x["baseline_stop_timestamp"]).astype(int), axis=1)


In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_spike_times"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: filter_spike_times(x["spike_times"], start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"]).astype(int), axis=1)


In [ ]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"].head()

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_stop_timestamp"].head()

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_spike_times"].iloc[0]

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_neuron_average_fr"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[1], axis=1)
TRIALS_AND_SPECTRAL_DF["baseline_neuron_average_timestamp"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[0], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_fr"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[1].T, axis=1)
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_timestamp"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[0], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=["spike_clusters", "spike_times", "neuron_average_fr", "neuron_average_timestamps",], errors="ignore")

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_fr"].iloc[0].shape

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_timestamp"].iloc[0].shape

# OLD Stuff

## Getting the ranges of each cluster

- Getting the index range

In [ ]:
list(TRIALS_AND_SPECTRAL_DF.columns)

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_and_post_kmeans_cluster"].iloc[0]

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_and_post_comp_id"] = TRIALS_AND_SPECTRAL_DF["trial_and_post_kmeans_cluster"].apply(lambda x: np.vectorize(cluster_to_comp_id.get)(x.astype(str)))

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_and_post_competitiveness"] = TRIALS_AND_SPECTRAL_DF["trial_and_post_kmeans_cluster"].apply(lambda x: np.vectorize(cluster_to_competitiveness.get)(x.astype(str)))

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_and_post_competitiveness"] 

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_and_post_competitiveness"].iloc[0].shape

In [ ]:
# TRIALS_AND_SPECTRAL_DF["cluster_index_ranges_dict"] = TRIALS_AND_SPECTRAL_DF["trial_and_post_comp_id"].apply(lambda x: find_consecutive_ranges(x, min_length=20))

TRIALS_AND_SPECTRAL_DF["cluster_index_ranges_dict"] = TRIALS_AND_SPECTRAL_DF["trial_and_post_competitiveness"].apply(lambda x: find_consecutive_ranges(x[:200], min_length=20))


In [ ]:
TRIALS_AND_SPECTRAL_DF["cluster_index_ranges_dict"].iloc[0]

In [ ]:
TRIALS_AND_SPECTRAL_DF.head()

In [ ]:
TRIALS_AND_SPECTRAL_DF["cluster_timestamp"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: x["video_timestamps"][x["trial_and_post_frame_index"]], axis=1)


- Calculating the times in milliseconds of each cluster frame

In [ ]:
TRIALS_AND_SPECTRAL_DF["cluster_times"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: (np.array(x["cluster_timestamp"]) - x["first_timestamp"]) // 20, axis=1)


- Updating the index to use cluster times and timestamps based on video frame timestamps list

In [ ]:
TRIALS_AND_SPECTRAL_DF["cluster_times_ranges_dict"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: update_tuples_in_dict(x["cluster_index_ranges_dict"], x["cluster_times"]), axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF["cluster_timestamps_ranges_dict"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: update_tuples_in_dict(x["cluster_index_ranges_dict"], x["cluster_timestamp"]), axis=1)

- Combining the win and loss label with the cluster

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_cluster_times_ranges_dict"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: {"{}_{}".format(x["trial_label"], k): v for k, v in x["cluster_times_ranges_dict"].items()}, axis=1)


In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_cluster_timestamps_ranges_dict"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: {"{}_{}".format(x["trial_label"], k): v for k, v in x["cluster_timestamps_ranges_dict"].items()}, axis=1)


In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_cluster_timestamps_ranges_dict"].iloc[0]

In [ ]:
TRIALS_AND_SPECTRAL_DF.columns

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_and_post_coherence_timestamps"].iloc[0]